In [1]:
import numpy as np
import yasa
from statsmodels.tsa import stattools
import pandas as pd

from scipy.integrate import simpson
from scipy.signal import welch

from config.constants import SAMPLING_FREQUENCY_HZ, SPECTRAL_BANDS, CHANNELS

In [2]:
N_CHANNELS = 2
# n_segs, seg_n_samples = 3, 5
n_segs, seg_n_samples = 3, 3105
# n_segs, seg_n_samples = 100_000, 3105

n_samples_edf = n_segs * seg_n_samples

In [3]:
segmented_sigs = np.zeros((n_segs, N_CHANNELS, seg_n_samples))
for chn in range(N_CHANNELS):
    sig = np.random.randint(-9, 10, n_samples_edf)
    # sig = np.arange(n_samples_edf)
    # np.random.shuffle(sig, )
    segmented_sigs[:, chn, :] = sig.reshape((n_segs, seg_n_samples))
ss = segmented_sigs  # alias
segmented_sigs

array([[[-5.,  2.,  9., ..., -5., -2., -9.],
        [-5., -7.,  0., ...,  2., -9.,  5.]],

       [[ 3., -2., -8., ..., -1.,  4., -4.],
        [ 7.,  3.,  4., ...,  8., -4.,  8.]],

       [[ 9.,  9., -5., ..., -6., -6., -7.],
        [-7., -7.,  9., ...,  1.,  0., -5.]]], shape=(3, 2, 3105))

We want to compute the four features for each segment (axis 0) as a batch.

# ✅ Variances

In [4]:
variances = segmented_sigs.var(axis=-1)
# Indexed as [seg, chn]
variances
#    ~ 2 sec for 100_000 segs
# - > ~20 sec for 1_000_000 segs

array([[30.24172699, 29.43441906],
       [31.15494836, 29.83666363],
       [30.54398096, 29.40201545]])

# Bandpower spectrum

In [5]:
# With yasa

# convert SPECTRAL_BANDS to list of tuples
bands = [(low, high, name) for name, (low, high) in SPECTRAL_BANDS.items()]

lowest_band_freq = bands[0][0]
win_sec = 2 / lowest_band_freq
for seg in range(n_segs):
    bandpowers = yasa.bandpower(segmented_sigs[seg], SAMPLING_FREQUENCY_HZ, CHANNELS, win_sec=win_sec, bands=bands,
                                relative=False)
bandpowers
# ~3:30 min for 100_000 segs
#>~33 min for 1_000_000 segs

,Delta,Theta,Alpha,Beta,Gamma,TotalAbsPow,FreqRes,Relative
Chan,,,,,,,,
EEG SQ_D-SQ_C,0.869717,1.275058,1.103443,6.102031,19.077943,28.662001,0.250038,False
EEG SQ_P-SQ_C,1.225290,1.241958,0.970875,6.624225,17.887676,28.316487,0.250038,False


In [6]:
from preprocessing.extract_features import bandpowers_batch

bandpowers = bandpowers_batch(segmented_sigs, SAMPLING_FREQUENCY_HZ, SPECTRAL_BANDS)

#   ~  20 sec 100_000 segs
# -> ~ 3 min for 1_000_000 segs

In [7]:
bandpowers.shape

(3, 2, 5)

In [8]:
seg = 2
pd.DataFrame(bandpowers[seg, :, :], index=CHANNELS, columns=SPECTRAL_BANDS.keys())

,Delta,Theta,Alpha,Beta,Gamma
EEG SQ_D-SQ_C,0.924260,1.301248,1.179034,6.387397,19.307458
EEG SQ_P-SQ_C,1.128337,1.156143,0.962426,6.676519,17.841984


# ACFW

In [9]:
def autocorrelation_function_width(sig: np.ndarray) -> int:
    autocorr = stattools.acf(sig, nlags=len(sig), fft=True)
    lag = np.abs(autocorr - 0.5).argmin()
    return lag

In [10]:
# Single seg
acfws = []
for chn in range(N_CHANNELS):
    acfws.append(autocorrelation_function_width(segmented_sigs[0, chn, :]))
acfws

[np.int64(243), np.int64(1127)]

In [11]:
# Not vectorized
acfws = np.zeros((n_segs, N_CHANNELS))
for seg in range(n_segs):
    for chn in range(N_CHANNELS):
        acfws[seg, chn] = autocorrelation_function_width(segmented_sigs[seg, chn, :])
#    ~ 50s for 100_000 segs
# -> ~ 8 min for 1_000_000 segs

In [12]:
# Equivalent to prior:
acfws = np.apply_along_axis(autocorrelation_function_width, -1, segmented_sigs)

In [13]:
acfws

array([[ 243, 1127],
       [ 717,  191],
       [ 325,   60]])

# ✅ Correlation Coefficient

In [23]:
# correct, but not vectorized solution
seg = 0
np.corrcoef(ss[seg, 0, :], ss[seg, 1, :])

array([[1.        , 0.00982675],
       [0.00982675, 1.        ]])

In [26]:
corrcoefs = np.array([np.corrcoef(ss[seg, 0, :], ss[seg, 1, :])[0, 1] for seg in range(n_segs)])
corrcoefs.shape
#    ~ 7s for 100_000 segs
# -> ~ 70 sec for 1_000_000segs

(3,)

In [16]:
def corrcoef_batch(segmented_sigs: np.ndarray) -> np.ndarray:
    # vectorized Pearson correlation per segment
    # Distal and proximal
    d = segmented_sigs[:, 0, :].astype(float)  # shape (n_segs, seg_n_samples)
    p = segmented_sigs[:, 1, :].astype(float)

    d_mean = d.mean(axis=1, keepdims=True)
    p_mean = p.mean(axis=1, keepdims=True)

    d_d = d - d_mean
    p_d = p - p_mean

    num = np.sum(d_d * p_d, axis=1)
    den = np.sqrt(np.sum(d_d ** 2, axis=1) * np.sum(y_d ** 2, axis=1))

    corrcoefs = num / np.where(den == 0, np.nan, den)  # shape (n_segs,)

In [17]:
# vectorized Pearson correlation per segment
# Distal and proximal
d = segmented_sigs[:, 0, :]  # shape (n_segs, seg_n_samples)
p = segmented_sigs[:, 1, :]

# Demean
d -= d.mean(axis=1, keepdims=True)
p -= p.mean(axis=1, keepdims=True)

num = np.sum(d * p, axis=1)
den = np.sqrt(np.sum(d ** 2, axis=1) * np.sum(p ** 2, axis=1))

my_corrcoefs = num / den  # shape (n_segs,)

In [18]:
my_corrcoefs.shape

(3,)

In [19]:
np.allclose(my_corrcoefs, corrcoefs)

True

# Combine Features

## Flatten Bandpowers

In [46]:
bps = []
for s in range(n_segs):
    seg = []
    for c in range(N_CHANNELS):
        chn = []
        for b in range(len(SPECTRAL_BANDS)):
            letter = "ABCDE"[b]
            chn.append(f"{s}{letter}{c}")
        seg.append(chn)
    bps.append(seg)
bps = np.array(bps)

print(bps.shape)
bps

(3, 2, 5)


array([[['0A0', '0B0', '0C0', '0D0', '0E0'],
        ['0A1', '0B1', '0C1', '0D1', '0E1']],

       [['1A0', '1B0', '1C0', '1D0', '1E0'],
        ['1A1', '1B1', '1C1', '1D1', '1E1']],

       [['2A0', '2B0', '2C0', '2D0', '2E0'],
        ['2A1', '2B1', '2C1', '2D1', '2E1']]], dtype='<U3')

In [47]:
print(bps.shape)
bps_flat = bps.reshape(n_segs, -1)
bps_flat.shape

(3, 2, 5)


(3, 10)

In [48]:
bps_flat

array([['0A0', '0B0', '0C0', '0D0', '0E0', '0A1', '0B1', '0C1', '0D1',
        '0E1'],
       ['1A0', '1B0', '1C0', '1D0', '1E0', '1A1', '1B1', '1C1', '1D1',
        '1E1'],
       ['2A0', '2B0', '2C0', '2D0', '2E0', '2A1', '2B1', '2C1', '2D1',
        '2E1']], dtype='<U3')

## Combine Arrays

In [51]:
for feat in [corrcoefs, acfws, variances, bps_flat]:
    print(feat.shape)

(3,)
(3, 2)
(3, 2)
(3, 10)


In [56]:
corrcoefs

array([ 0.00982675,  0.00650648, -0.02297745])

In [60]:
corrcoefs_expanded = np.expand_dims(list(corrcoefs), 1)
# corrcoefs_expanded = corrcoefs[:, None]
corrcoefs_expanded

array([[ 0.00982675],
       [ 0.00650648],
       [-0.02297745]])

In [59]:
stacked = np.hstack([
    corrcoefs_expanded,
    acfws,
    variances,
    bps_flat,
])
stacked

array([['0.009826754742833028', '243', '1127', '30.24172699479568',
        '29.434419058139564', '0A0', '0B0', '0C0', '0D0', '0E0', '0A1',
        '0B1', '0C1', '0D1', '0E1'],
       ['0.00650648451041026', '717', '191', '31.154948358706672',
        '29.836663632756892', '1A0', '1B0', '1C0', '1D0', '1E0', '1A1',
        '1B1', '1C1', '1D1', '1E1'],
       ['-0.02297744794395503', '325', '60', '30.543980956381713',
        '29.402015449602086', '2A0', '2B0', '2C0', '2D0', '2E0', '2A1',
        '2B1', '2C1', '2D1', '2E1']], dtype='<U32')

In [61]:
for i, c in enumerate('DP'):
    print(i,c)

0 D
1 P
